In [ ]:
from helpers_resnet import *

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
plt.rcParams['figure.figsize'] = (18,8)
font = {'family' : 'DejaVu Sans',  'weight' : 'normal',  'size'  : 20}
plt.rc('font', **font)


In [ ]:
resnet = torchvision.models.resnet50(pretrained=True)
adaptive_pooling = AdaptiveConcatPool2d()
head = Head()
resnet.avgpool = adaptive_pooling
resnet.fc = head

In [ ]:
resnet = resnet.cuda()

In [ ]:
resnet = nn.DataParallel(resnet)

In [ ]:
checkpoint = torch.load('save_freezed_6.pth')
resnet.load_state_dict(checkpoint['state_dict'])
print('loading pretrained freezed model!')

## remove it from parallel

In [ ]:
resnet = resnet.module

In [ ]:
resnet

In [ ]:
df_valid = pd.read_csv('./train-valid-splits-video/valid.csv')

In [ ]:
bs = 32

In [ ]:
valid_loader = load_data(df_valid, bs, 1)

In [ ]:
resnet.eval()
y_true = []
y_pred = []
with torch.no_grad():
    for X, y in valid_loader:
        X = X.to(device)
        y = Variable(y.float()).to(device) 
        X = X.permute(0,2,1,3,4)
        y = y.squeeze(dim=1)
        y = y.float()
        output = resnet(X)
        y = y.detach().cpu()
        #loss = criterion(output, y)
        preds = torch.sigmoid(output)
        preds = preds.to(torch.float32) 
        preds = preds.detach().cpu()
        y_pred.append(preds)
        y_true.append(y)

In [ ]:
y_tr = np.vstack([t.__array__() for tensor in y_true for t in tensor])

In [ ]:
y_pr = np.vstack([t.__array__() for tensor in y_pred for t in tensor])

In [ ]:
thresholds = nsea_compute_thresholds(y_tr, y_pr)

In [ ]:
thresholds

In [ ]:
acc = accuracy_score(y_tr, y_pr)
f1 = f1_score(y_tr, y_pr, average="samples")

In [ ]:
new_compute_metrics(y_tr, y_pr, thresholds)